# Step 2

In this notebook we trigger a one-off export of data from MySQL to Neptune via S3. We use an AWS Glue job to export product, product category and supplier data to CSV files, which the job copies to S3. We then use the Neptune bulk load API to load the data from S3 into Neptune.

For details on converting from a relational data model to a graph data model, see [Converting a Relational Data Model to a Graph Model](https://github.com/aws-samples/aws-dbs-refarch-graph/tree/master/src/converting-to-graph#converting-a-relational-data-model-to-a-graph-model).

<img src="https://s3.amazonaws.com/aws-neptune-customer-samples/neptune-sagemaker/images/mysql-2-neptune-02.png"/>

# Export from MySQL to S3

First, we'll export some 'static' dimension data (product, product_category, supplier) from MySQL to S3 using a Glue job, `export_from_mysql_to_s3`, the script for which can be found [here](https://github.com/aws-samples/amazon-neptune-samples/tree/master/gremlin/glue-neptune/glue-jobs/mysql-neptune/export-from-mysql-to-s3.py). The script uses the [glue_neptune](https://github.com/awslabs/amazon-neptune-tools/tree/master/glue-neptune) Python library.

The export creates CSV files formatted according to the Amazon Neptune [bulk load format](https://docs.aws.amazon.com/neptune/latest/userguide/bulk-load-tutorial-format-gremlin.html).

In [ ]:
%run './glue_utils.py'
%run '../util/neptune.py'

In [ ]:
job_name = glue_resource('export_from_mysql_to_s3')

In [ ]:
run_job(job_name)

# Import from S3

Next, we'll trigger Neptune's bulk load API, and load the graph using the CSV data in S3. (We're clearing the graph here before beginning the import on the assumption that this is the first step in a migration to a fresh Neptune database.)

The import creates Product and Supplier vertices and connects them with SUPPLIER edges. (A supplier supplies many products.)

In [ ]:
neptune.clear()
neptune.bulkLoad(source=os.environ['S3_EXPORT_PATH'])

## Query the graph

Let's query the graph to review the results of the import.

In [ ]:
g = neptune.graphTraversal()

### Count the number of Product and Supplier vertices in the graph

In [ ]:
print('# Products : {}'.format(g.V().hasLabel('Product').count().next()))
print('# Suppliers: {}'.format(g.V().hasLabel('Supplier').count().next()))

### Show the products supplied by a supplier

Product vertices are connected to Supplier vertices using an outgoing SUPPLIER edge.

In [ ]:
results = (g.V().hasLabel('Supplier').limit(1).
      project('supplier', 'products').
      by('name').
      by(in_('SUPPLIER').values('name').fold()).next())

print('Supplier: {}\nProducts: {}'.format(results['supplier'], results['products']))